# 뉴스 편향성 분석 모델

이 노트북은 뉴스 기사의 민주당과 국힘에 대한 편향성을 분석하는 딥러닝 모델을 구현합니다.

In [ ]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import (
    AutoTokenizer, 
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding
)
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

## 1. 데이터 로드 및 전처리

In [ ]:
# 데이터 로드
df1 = pd.read_csv('../data/labeling1.csv')
df2 = pd.read_csv('../data/labeling2.csv')
df = pd.concat([df1, df2], ignore_index=True)

In [ ]:
# 스탠스 레이블 매핑
stance_mapping = {'우호': 0, '중립': 1, '비판': 2}

# 민주당과 국힘 스탠스 레이블 변환
df['민주당_스탠스_레이블'] = df['민주당_스탠스'].map(stance_mapping)
df['국힘_스탠스_레이블'] = df['국힘_스탠스'].map(stance_mapping)

# # 텍스트 데이터 준비
# df['text'] = df['title_cleaned'] + ' ' + df['content_cleaned']

print(f"전체 데이터 수: {len(df)}")
print("\n민주당 스탠스 분포:")
print(df['민주당_스탠스'].value_counts())
print("\n국힘 스탠스 분포:")
print(df['국힘_스탠스'].value_counts())

## 2. 데이터셋 클래스 정의

In [ ]:
class NewsDataset(Dataset):
    def __init__(self, texts, dem_labels, ppp_labels, tokenizer, max_length=512):
        self.texts = texts
        self.dem_labels = dem_labels
        self.ppp_labels = ppp_labels
        self.tokenizer = tokenizer
        self.max_length = max_length
        
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = str(self.texts[idx])
        encoding = self.tokenizer(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'dem_label': torch.tensor(self.dem_labels[idx], dtype=torch.long),
            'ppp_label': torch.tensor(self.ppp_labels[idx], dtype=torch.long)
        }

## 3. 모델 정의

In [ ]:
class NewsBiasModel(torch.nn.Module):
    def __init__(self, model_name, num_labels=3):
        super().__init__()
        # 기본 모델 로드
        self.bert = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)
        
        # 드롭아웃 레이어 추가
        self.dropout = torch.nn.Dropout(0.2)  # 드롭아웃 비율 증가
        
        # 공통 특성 추출을 위한 레이어
        self.shared_layer = torch.nn.Linear(num_labels, 256)
        self.activation = torch.nn.ReLU()
        
        # 각 정당별 분류기
        self.classifier_dem = torch.nn.Sequential(
            torch.nn.Linear(256, 128),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.1),
            torch.nn.Linear(128, num_labels)
        )
        
        self.classifier_ppp = torch.nn.Sequential(
            torch.nn.Linear(256, 128),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.1),
            torch.nn.Linear(128, num_labels)
        )
        
    def forward(self, input_ids, attention_mask, dem_label=None, ppp_label=None):
        # BERT 출력
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.logits
        
        # 공통 특성 추출
        shared_features = self.dropout(pooled_output)
        shared_features = self.shared_layer(shared_features)
        shared_features = self.activation(shared_features)
        
        # 각 정당별 예측
        dem_logits = self.classifier_dem(shared_features)
        ppp_logits = self.classifier_ppp(shared_features)
        
        if dem_label is not None and ppp_label is not None:
            loss_fct = torch.nn.CrossEntropyLoss()
            dem_loss = loss_fct(dem_logits, dem_label)
            ppp_loss = loss_fct(ppp_logits, ppp_label)
            loss = dem_loss + ppp_loss
            return {'loss': loss, 'dem_logits': dem_logits, 'ppp_logits': ppp_logits}
        
        return {'dem_logits': dem_logits, 'ppp_logits': ppp_logits}

## 4. 학습 준비

In [ ]:
# 데이터 분할
train_texts, val_texts, train_dem_labels, val_dem_labels, train_ppp_labels, val_ppp_labels = train_test_split(
    df['text'].values,
    df['민주당_스탠스_레이블'].values,
    df['국힘_스탠스_레이블'].values,
    test_size=0.2,
    random_state=42
)

# 토크나이저 초기화
tokenizer = AutoTokenizer.from_pretrained('beomi/KcELECTRA-base')

# 데이터셋 생성
train_dataset = NewsDataset(train_texts, train_dem_labels, train_ppp_labels, tokenizer)
val_dataset = NewsDataset(val_texts, val_dem_labels, val_ppp_labels, tokenizer)

# 모델 초기화
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = NewsBiasModel('beomi/KcELECTRA-base').to(device)

# 커스텀 데이터 콜레이터
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
# 커스텀 평가 함수
def compute_metrics(eval_pred):
    dem_preds, ppp_preds = eval_pred.predictions
    dem_labels, ppp_labels = eval_pred.label_ids
    
    dem_preds = np.argmax(dem_preds, axis=1)
    ppp_preds = np.argmax(ppp_preds, axis=1)
    
    dem_report = classification_report(dem_labels, dem_preds, target_names=['우호', '중립', '비판'], output_dict=True)
    ppp_report = classification_report(ppp_labels, ppp_preds, target_names=['우호', '중립', '비판'], output_dict=True)
    
    return {
        'dem_f1': dem_report['weighted avg']['f1-score'],
        'ppp_f1': ppp_report['weighted avg']['f1-score']
    }

In [ ]:
# 학습 인자 설정
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=10,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=1e-5,
    warmup_ratio=0.1,
    weight_decay=0.05,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
    metric_for_best_model='dem_f1',
    gradient_accumulation_steps=4,  # 그래디언트 누적
    fp16=True,  # 혼합 정밀도 학습
    label_smoothing_factor=0.1,  # 레이블 스무딩
    optim='adamw_torch'  # AdamW 옵티마이저 사용
)

# 트레이너 초기화
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

## 5. 학습 및 평가

In [ ]:
# 학습 실행
trainer.train()

# 최종 평가
final_metrics = trainer.evaluate()
print("\n최종 평가 결과:")
print(f"민주당 F1 점수: {final_metrics['eval_dem_f1']:.4f}")
print(f"국힘 F1 점수: {final_metrics['eval_ppp_f1']:.4f}")

## 6. 모델 저장 및 로드

In [ ]:
# 모델 저장
trainer.save_model('./best_model')
tokenizer.save_pretrained('./best_model')

# 모델 로드
def load_model(model_path):
    model = NewsBiasModel('beomi/KcELECTRA-base')
    model.load_state_dict(torch.load(f'{model_path}/pytorch_model.bin'))
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    return model, tokenizer

## 7. 새로운 기사에 대한 예측

In [ ]:
def predict_stance(text, model, tokenizer, device):
    model.eval()
    encoding = tokenizer(
        text,
        add_special_tokens=True,
        max_length=512,
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )
    
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        dem_pred = torch.argmax(outputs['dem_logits'], dim=1).item()
        ppp_pred = torch.argmax(outputs['ppp_logits'], dim=1).item()
    
    return dem_pred, ppp_pred

In [ ]:
def predict_csv_file(csv_path, model, tokenizer, device):
    # CSV 파일 로드
    df = pd.read_csv(csv_path)
    
    # 예측 결과를 저장할 리스트
    dem_predictions = []
    ppp_predictions = []
    
    # 각 텍스트에 대해 예측 수행
    for text in tqdm(df['text'], desc="Predicting"):
        dem_pred, ppp_pred = predict_stance(text, model, tokenizer, device)
        dem_predictions.append(dem_pred)
        ppp_predictions.append(ppp_pred)
    
    # 예측 결과를 DataFrame에 추가
    df['민주당_스탠스_레이블'] = dem_predictions
    df['국힘_스탠스_레이블'] = ppp_predictions
    
    # 예측 결과를 숫자에서 텍스트로 변환
    reverse_mapping = {v: k for k, v in stance_mapping.items()}
    df['민주당_스탠스'] = df['민주당_스탠스_레이블'].map(reverse_mapping)
    df['국힘_스탠스'] = df['국힘_스탠스_레이블'].map(reverse_mapping)
    
    # 결과 저장
    output_path = csv_path.replace('.csv', '_predicted.csv')
    df.to_csv(output_path, index=False)
    
    # 예측 결과 통계 출력
    print(f"\n{csv_path} 예측 결과:")
    print("\n민주당 스탠스 예측 분포:")
    print(df['민주당_스탠스'].value_counts())
    print("\n국힘 스탠스 예측 분포:")
    print(df['국힘_스탠스'].value_counts())
    
    return df

In [ ]:
# 모델 로드
model, tokenizer = load_model('./best_model')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# CSV 파일 예측
csv_path = "./data/전체1.csv"  # 여기에 실제 CSV 파일 경로를 입력하시면 됩니다
predicted_df = predict_csv_file(csv_path, model, tokenizer, device)
predicted_df.head()

In [ ]:
# 저장
predicted_df.to_csv(csv_path, index=False)